In [ ]:
from tqdm import tqdm

from User import User
from Survey import Survey
from Intake import Intake

from utils import *

import pandas as pd

import nest_asyncio
import asyncio

# Only run nest_asyncio in a Jupyter Notebook environment
nest_asyncio.apply()

In [ ]:
# Load the respective files

async def get_all_dataframes():
    return await asyncio.gather(
        load_intake24(),
        load_heifa_ingredients(),
        load_heifa_recipes()
    )

intake24_df, heifa_food_df, heifa_recipes_df = asyncio.run(get_all_dataframes())

# Breakdown of Intake 24:

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [ ]:
user_dict = create_user_objects(intake24_df)

for user_id in user_dict.keys():

    print(f"Printing for User {user_id}")
    user_obj = user_dict[user_id]
    user_obj.print_information()

# Breakdown of HEIFA (Food Composition)

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [ ]:
# Create the objects
food_composition_dict = create_food_objects(heifa_food_df)

for key, food_comp_obj in food_composition_dict.items():

    food_comp_obj.print_full_details()

# Breakdown of HEIFA (Recipes)

- Every recipe has multiple ingredients
- Keys are repeated across rows (similar to Survey ID of Intake24)
- Every ingredient has respective proportion to the recipe

In [ ]:
recipe_dict = create_recipe_objects(heifa_recipes_df)

for id, recipe_obj in recipe_dict.items():
    print(f"Printing for ID {id}\n")
    recipe_obj.print_ingredients_information()

## Mapping between Intake24 and HEIFA Ingredients

- For each user, extract the given nutrients and store in an array.
- This is from ALL the survey data.
- We don't care about the order here.
- The array will contain a list of dictionaries/JSON.

In the array:

- Use the HEIFA ID (from user) to map to the HEIFA Ingredients' HEIFA ID.
- Check if a result is found or not.
- Check if it requires a recipe or not.

## Mapping between Intake24 and HEIFA Recipes

This is in case a recipe is found (The second step).

- For the given recipe, extract the given nutrients ID and proportion, store in an array.
- We don't care about the order here.
- The array will contain a list of dictionaries.

In the array:

- Use the HEIFA ID (from the recipes) to map the HEIFA Ingredients' HEIFA ID.
- Check the energy and serving size.

In [ ]:
user_meals = fetch_user_food_list(user_dict)

In [ ]:
for user_id, meals_list in user_meals.items():

    print(f"Printing for user {user_id}")
    calculate_portion_serving_heifa(meals_list, food_composition_dict, recipe_dict)
    print("\n\n")

In [ ]:
food_list = user_meals[1]

# For no food groups, we skip the serving size calculation

for food_dict in food_list:

    # Get the ID and the food object
    for heifa_id, food_obj in food_dict.items():
    
        heifa_obj = food_composition_dict[heifa_id]

        print(f"HEIFA ID: {heifa_id}\n")
        print(f"Portion size (gram): {food_obj.portion_size}")
        print(f"Portion size (energy with fibre): {food_obj.energy_with_fibre}")
        print(f"Is it a recipe: {heifa_obj.is_recipe}\n")
        
        print(f"HEIFA Serving size: {heifa_obj.serving_size}")
        print(f"HEIFA Serving measure: {heifa_obj.serving_measure}\n")

        # Skip the ones that have no food group
        if not heifa_obj.required_portion_calculation:
            print("*" * 20)
            print("\n\n")
            continue

        if heifa_obj.is_recipe:

            # Get the recipe ID via 8 digit code
            size = food_obj.portion_size
            recipe_obj = recipe_dict[heifa_obj.eight_digit_code]
            recipe_pieces = recipe_obj.recipe_pieces

            for heifa_id, ingredient_obj in recipe_pieces.items():

                # Find the object
                pieced_heifa = food_composition_dict[heifa_id]
                print(f"{heifa_id} -> Recipe: {pieced_heifa.is_recipe}")

                # Break down the proportion first
                pieced_portion = round(size * ingredient_obj.proroption, 1)

                # Energy convertion
                energy_amount = round((pieced_portion * ingredient_obj.energy_with_fibre) / 100, 2)

                print(f"Portion of {size} with proportion {ingredient_obj.proroption}: {pieced_portion}")
                print(f"Energy per 100g: {ingredient_obj.energy_with_fibre} (For {pieced_portion}g: {energy_amount})")

                # Skip the ones that have no food group
                #if not pieced_heifa.required_portion_calculation:
                #    continue
                    
                #serving_size = pieced_heifa.calculate_serving_size(energy_amount, pieced_portion)
                #print(f"Recommended serving size: {serving_size}\n")
                
                print("\n")
                
            print("*" * 20)
            print("\n\n")
            continue

        recommended_serving_size = heifa_obj.calculate_serving_size(food_obj.energy_with_fibre, food_obj.portion_size)
        print(f"Recommended serving size: {recommended_serving_size}\n")

        print("*" * 20)
        print("\n\n")
        break